`Course Instructor`: **John Chiasson**

`Author (TA)`: **Ruthvik Vaila**

# Notes:
* In this notebook we shall use `.tfrecord` files to train a model.
* While the model is training keep an eye on the time taken and RAM usage of your machine.
* Tested on `Python 3.7.5` with `Tensorflow 1.15.0` and `Keras 2.2.4`. 
* Tested on `Python 2.7.17` with `Tensorflow 1.15.3` and `Keras 2.2.4`.

# Imports

In [1]:
import sys, os
sys.version

'3.7.5 (default, Nov  7 2019, 10:50:52) \n[GCC 8.3.0]'

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import sys, pickle, os, keras
import h5py, time, inspect, IPython
import IPython.display as display
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

Using TensorFlow backend.


In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9554650987531913927,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16062015043385408398
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9175293818520174154
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7400190772
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14409573039205546539
 physical_device_desc: "device: 0, name: GeForce RTX 2080 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
tf.__version__

'1.15.0'

In [5]:
keras.__version__

'2.2.4'

# Parsing all the `.tfrecords` and make an iterable dataset 

In [6]:
def parser(record):
    image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.VarLenFeature(tf.float32),
    }
    parsed = tf.io.parse_single_example(record, image_feature_description)
    
    image = parsed['image_raw']
    image = tf.sparse.to_dense(image,default_value = 0)
    label = tf.cast(parsed["label"], tf.int32)
    
    #return {"image_data": image}, label
    return image, label
    

In [7]:
def load_dataset(path='', shuffle_buffer_size=None, batch_size=None, compression='GZIP'):
    filenames = [file for file in os.listdir(os.path.join(os.getcwd(), path)) if file.endswith('.tfrecord')]
    filenames = [os.path.join(path, file) for file in filenames]
    print('.tfrecord files:{}'.format(filenames))
    raw_image_dataset = tf.data.TFRecordDataset(filenames, compression_type=compression)
    print('\n raw data:{}'.format(raw_image_dataset))
    parsed_image_dataset = raw_image_dataset.map(parser)
    print('\n parsed data:{}'.format(parsed_image_dataset))
    final_dataset = parsed_image_dataset.shuffle(shuffle_buffer_size).batch(batch_size)
    print('\n final dataset:{}'.format(final_dataset))

    return final_dataset

* Take 10 images and inspect but only works in eager mode enbale it by adding `tf.compat.v1.enable_eager_execution()` to the `Imports` cell. 

`for item in parsed_image_dataset.take(10):
    print
    print('Image Data')
    print(item[0])
    print(item[0].numpy().sum())
    print
    print('Label Data')
    print(item[1])
    print(item[1].numpy())`

# Load  `.tfrecords` 
* Load `.tfrecords` for the training data
* See more about [shuffle_buffer_size](https://stackoverflow.com/questions/46444018/meaning-of-buffer-size-in-dataset-map-dataset-prefetch-and-dataset-shuffle)

In [8]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = BATCH_SIZE**2

#train_dataset = load_dataset('40_tfrecords/train', SHUFFLE_BUFFER_SIZE, BATCH_SIZE) #
train_dataset = load_dataset('parallel_tfrecords/train', SHUFFLE_BUFFER_SIZE, BATCH_SIZE)

.tfrecord files:['40_tfrecords/train/EMNIST_train_strings_4.tfrecord', '40_tfrecords/train/EMNIST_train_strings_34.tfrecord', '40_tfrecords/train/EMNIST_train_strings_27.tfrecord', '40_tfrecords/train/EMNIST_train_strings_38.tfrecord', '40_tfrecords/train/EMNIST_train_strings_9.tfrecord', '40_tfrecords/train/EMNIST_train_strings_12.tfrecord', '40_tfrecords/train/EMNIST_train_strings_6.tfrecord', '40_tfrecords/train/EMNIST_train_strings_37.tfrecord', '40_tfrecords/train/EMNIST_train_strings_24.tfrecord', '40_tfrecords/train/EMNIST_train_strings_25.tfrecord', '40_tfrecords/train/EMNIST_train_strings_32.tfrecord', '40_tfrecords/train/EMNIST_train_strings_26.tfrecord', '40_tfrecords/train/EMNIST_train_strings_13.tfrecord', '40_tfrecords/train/EMNIST_train_strings_15.tfrecord', '40_tfrecords/train/EMNIST_train_strings_35.tfrecord', '40_tfrecords/train/EMNIST_train_strings_16.tfrecord', '40_tfrecords/train/EMNIST_train_strings_3.tfrecord', '40_tfrecords/train/EMNIST_train_strings_5.tfrecord'

# Train a small NN model using `tf.keras.model.fit` 

In [9]:
def smol_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1500, input_dim=3630, activation='relu'),
        tf.keras.layers.Dense(47)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.005),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['sparse_categorical_accuracy'])
    return model
    

In [10]:
model = smol_model()
model.summary()
history = model.fit(train_dataset, epochs=3)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1500)              5446500   
_________________________________________________________________
dense_1 (Dense)              (None, 47)                70547     
Total params: 5,517,047
Trainable params: 5,517,047
Non-trainable params: 0
_________________________________________________________________
Train on None steps
Epoch 1/3
3525/3525 [==============================] - 26s 7ms/step - loss: 1.5530 - sparse_categorical_accuracy: 0.6373
Epoch 2/3
3525/3525 [==============================] - 25s 7ms/step - loss: 1.2584 - sparse_categorical_accuracy: 0.6979
Epoch 3/3
3525/3525 [==============================] - 27s 8ms/step - loss: 1.1604 - sparse_categorical_accuracy: 0.7202


# Test the model

In [11]:
test_dataset = load_dataset('40_tfrecords/test', SHUFFLE_BUFFER_SIZE, BATCH_SIZE)
model.evaluate(test_dataset)

.tfrecord files:['40_tfrecords/test/EMNIST_test_strings_0.tfrecord', '40_tfrecords/test/EMNIST_test_strings_1.tfrecord']

 raw data:<TFRecordDatasetV1 shapes: (), types: tf.string>

 parsed data:<DatasetV1Adapter shapes: ((?,), ()), types: (tf.float32, tf.int32)>

 final dataset:<DatasetV1Adapter shapes: ((?, ?), (?,)), types: (tf.float32, tf.int32)>
    588/Unknown - 4s 6ms/step - loss: 1.4480 - sparse_categorical_accuracy: 0.6928

[1.4479922063800754, 0.6928028]

# Restart the notebook to free up the `GPU` and `RAM`

In [12]:
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}

# Exercise: Log the RAM usage for the two cases.
* How the RAM usage varies when training the model using `.tfrecord` vs direct `NumPy` array.
* How does the speed vary?
* Plot various metrics like `training cost vs epochs`, `training accuracy vs epochs` etc. These metrics can be found in the dictionary `history` returned by the model.